In [5]:
import angr, claripy
from pwn import *

In [6]:
!objdump -d p3


p3:     file format elf64-x86-64


Disassembly of section .init:

0000000000001000 <_init>:
    1000:	48 83 ec 08          	sub    $0x8,%rsp
    1004:	48 8b 05 c5 2f 00 00 	mov    0x2fc5(%rip),%rax        # 3fd0 <__gmon_start__@Base>
    100b:	48 85 c0             	test   %rax,%rax
    100e:	74 02                	je     1012 <_init+0x12>
    1010:	ff d0                	call   *%rax
    1012:	48 83 c4 08          	add    $0x8,%rsp
    1016:	c3                   	ret

Disassembly of section .plt:

0000000000001020 <printf@plt-0x10>:
    1020:	ff 35 ca 2f 00 00    	push   0x2fca(%rip)        # 3ff0 <_GLOBAL_OFFSET_TABLE_+0x8>
    1026:	ff 25 cc 2f 00 00    	jmp    *0x2fcc(%rip)        # 3ff8 <_GLOBAL_OFFSET_TABLE_+0x10>
    102c:	0f 1f 40 00          	nopl   0x0(%rax)

0000000000001030 <printf@plt>:
    1030:	ff 25 ca 2f 00 00    	jmp    *0x2fca(%rip)        # 4000 <printf@GLIBC_2.2.5>
    1036:	68 00 00 00 00       	push   $0x0
    103b:	e9 e0 ff ff ff       	jmp    1020 <_init+0x20>

D

In [45]:
elf = ELF("./p3")

In [46]:
p = angr.Project("./p3", auto_load_libs=False, main_opts={'base_addr': 0x0})

In [52]:
class input_returner(angr.SimProcedure):
    def run(self, inval):
        return inval * 3

p.hook_symbol('killer', input_returner())

WARNING  | 2024-06-13 20:24:16,743 | angr.project   | Address is already hooked, during hook(0x1139, <SimProcedure input_returner>). Re-hooking.


4409

In [53]:
s = p.factory.entry_state(addr=elf.symbols["main"])
sm = p.factory.simgr(s)

In [54]:
results = sm.explore(find=lambda s: b"Good" in s.posix.dumps(1), avoid=lambda s: b"Bad" in s.posix.dumps(1))

In [55]:
results

<SimulationManager with 1 found>

In [56]:
results.found[0].posix.stdout.concretize()

[b'Good job!']